In [1]:
import os
from pathlib import Path

import json
import pandas as pd
from rdkit import Chem

cwd = os.getcwd()
if cwd.endswith('notebook/PG'):
    os.chdir('../..')
    cwd = os.getcwd()

from src.pg_modelling.ligand_utils import (
    generate_ccd_from_smiles, 
    generate_ccd_from_mol, 
    sanitize_ligand_name, 
    generate_conformation,
)
from src.pg_modelling.af3.scoring import process_af3_ligand_pulldown_results
from src.pg_modelling.protenix.scoring import process_protenix_ligand_pulldown_results

In [2]:
data_folder = Path(os.path.expanduser('~')) / 'Documents' / 'PG_modelling'
assert data_folder.is_dir()

In [3]:
glycan_modifications = pd.read_csv(
    data_folder / 'glycan_modifications' / 'glycan_modifications.csv', 
    index_col='ligand_name',
)
print(f'Number of fragments in subset: {len(glycan_modifications):,}')
glycan_modifications

Number of fragments in subset: 9


,Name,Synonym,Formula,Monoisotopic Mass,Modifications,Degree Amidation,Degree Acetylation,Ontology,PGN Units,Glycan Units,...,Peptide,SMILES,INCHIKEY,clogP,RT,[M+H]+,[M+Na]+,[M+K]+,[M+2H]2+,[M+3H]3+
ligand_name,,,,,,,,,,,,,,,,,,,,,
G-NAM,(G)(NAM)-None,GlcN.MurNAc--None,C17H30N2O12,454.17987,Amidase,0,1,G2,1,2,...,NaN,CC(=O)NC1C(O)OC(CO)C(OC2OC(CO)C(O)C(O)C2N)C1OC...,CCRWBXATDKHQDG-UHFFFAOYSA-N,-4.7896,0,455.18715,477.16909,493.14303,228.09722,152.40057
G-DAM,(G)(DAM)-None,GlcN.MurNAcOAc--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(O)OC(COC(C)=O)C(OC2OC(CO)C(O)C(O)C2...,WMTUYCBSSKTNNR-UHFFFAOYSA-N,-4.2188,0,497.19772,519.17966,535.15360,249.10250,166.40409
NAG-MUR,(NAG)(MUR)-None,GlcNAc.MurN--None,C17H30N2O12,454.17987,Amidase,0,1,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)O)OC(C...,IGYLVGQPIFKPAO-UHFFFAOYSA-N,-4.7896,0,455.18715,477.16909,493.14303,228.09722,152.40057
DAG-MUR,(DAG)(MUR)-None,GlcNAcOAc.MurN--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)O)OC(C...,ZSDDRWVIQWSQPJ-UHFFFAOYSA-N,-4.2188,0,497.19772,519.17966,535.15360,249.10250,166.40409
NAG-NAM,(NAG)(NAM)-None,GlcNAc.MurNAc--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,WQKIVDUWFNRJHE-UHFFFAOYSA-N,-4.6122,0,497.19772,519.17966,535.15360,249.10250,166.40409
DAG-NAM,(DAG)(NAM)-None,GlcNAcOAc.MurNAc--None,C21H34N2O14,538.20100,Amidase,0,3,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,SUTALUMKMRSKIF-UHFFFAOYSA-N,-4.0414,0,539.20828,561.19022,577.16416,270.10778,180.40761
NAG-DAM,(NAG)(DAM)-None,GlcNAc.MurNAcOAc--None,C21H34N2O14,538.20100,Amidase,0,3,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,GKOAKWSMCSQKMH-UHFFFAOYSA-N,-4.0414,0,539.20828,561.19022,577.16416,270.10778,180.40761
DAG-DAM,(DAG)(DAM)-None,GlcNAcOAc.MurNAcOAc--None,C23H36N2O15,580.21157,Amidase,0,4,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,UGHTWFWJTFCKTB-UHFFFAOYSA-N,-3.4706,0,581.21885,603.20079,619.17473,291.11307,194.41114
G-MUR,(G)(MUR)-None,GlcN.MurN--None,C15H28N2O11,412.16931,Amidase,0,0,G2,1,2,...,NaN,CC(OC1C(N)C(O)OC(CO)C1OC1OC(CO)C(O)C(O)C1N)C(=O)O,UBADYJPVCNDSPI-UHFFFAOYSA-N,-4.9670,0,413.17659,435.15853,451.13247,207.09194,138.39705


In [5]:
ligand_folder = data_folder / 'glycan_modifications' / 'pdb'
ligand_folder.mkdir(exist_ok=True)

for ligand_name, row in glycan_modifications.iterrows():
    raw_name = row['Name']
    smiles = row['SMILES']

    mol = Chem.MolFromSmiles(smiles)
    try:
        mol = generate_conformation(mol)
    except ValueError:
        print(f'Error for ligand: {raw_name}')
        raise

    output_path = ligand_folder / f'{ligand_name}.pdb'
    Chem.MolToPDBFile(mol, output_path.as_posix())

[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:09] Molecule does not have explicit Hs. Consider calling 

## Protenix results

In [8]:
protenix_lysozyme_folder = data_folder / 'glycan_modifications' / 'protenix__lysozyme_pulldown'
protenix_results_df = process_protenix_ligand_pulldown_results('Human_Lysozyme', protenix_lysozyme_folder, run_posebusters=True)
protenix_results_df

folder  \
protein_name   ligand_name                                     
Human_Lysozyme G-MUR          Human_Lysozyme_C_P61626__G-MUR   
               NAG-MUR      Human_Lysozyme_C_P61626__NAG-MUR   
               DAG-MUR      Human_Lysozyme_C_P61626__DAG-MUR   
               NAG-DAM      Human_Lysozyme_C_P61626__NAG-DAM   
               DAG-DAM      Human_Lysozyme_C_P61626__DAG-DAM   
               G-NAM          Human_Lysozyme_C_P61626__G-NAM   
               DAG-NAM      Human_Lysozyme_C_P61626__DAG-NAM   
               NAG-NAM      Human_Lysozyme_C_P61626__NAG-NAM   
               G-DAM          Human_Lysozyme_C_P61626__G-DAM   

                                                               structure_file  \
protein_name   ligand_name                                                      
Human_Lysozyme G-MUR        /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-MUR      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               DAG-MUR      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-DAM      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               DAG-DAM      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               G-NAM        /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               DAG-NAM      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-NAM      /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               G-DAM        /Users/rs1521/Documents/PG_modelling/glycan_mo...   

                                seed       ptm      iptm  confidence  \
protein_name   ligand_name                                             
Human_Lysozyme G-MUR        seed_998  0.954696  0.986968    0.980513   
               NAG-MUR      seed_201  0.958441  0.979361    0.975177   
               DAG-MUR      seed_201  0.959054  0.979086    0.975080   
               NAG-DAM      seed_201  0.958758  0.975488    0.972142   
               DAG-DAM      seed_862  0.959768  0.974485    0.971542   
               G-NAM        seed_998  0.963097  0.971376    0.969720   
               DAG-NAM      seed_998  0.955708  0.967946    0.965498   
               NAG-NAM      seed_862  0.960965  0.965639    0.964704   
               G-DAM        seed_862  0.965220  0.978546    0.975880   

                            posebusters_score           posebusters_errors  
protein_name   ligand_name                                                  
Human_Lysozyme G-MUR                       20                         None  
               NAG-MUR                     20                         None  
               DAG-MUR                     20                         None  
               NAG-DAM                     20                         None  
               DAG-DAM                     20                         None  
               G-NAM                       20                         None  
               DAG-NAM                     20                         None  
               NAG-NAM                     20                         None  
               G-DAM                       19  minimum_distance_to_protein

## AF3

In [ ]:
ligand_folder_af3 = data_folder / 'glycan_modifications' / 'af3__inputs'
ligand_folder_af3.mkdir(exist_ok=True)

for ligand_name, row in glycan_modifications.iterrows():
    raw_name = row['Name']
    smiles = row['SMILES']

    ccd_code = f'LIG-PG-{ligand_name}'.replace('_', '-')
    ligand_seq = {
        'ligand': {
            'id': 'B',
            'ccdCodes': [ccd_code],
        }
    }
    try:
        ccd_data = generate_ccd_from_smiles(smiles, ccd_code)
    except ValueError:
        print(f'Error for ligand: {raw_name}')
        raise

    data = {
        'name': ligand_name,
        'sequences': [ligand_seq],
        'userCCD': ccd_data,
        'dialect': 'alphafold3',
        'version': 1,
    }
    with (ligand_folder_af3 / f'{ligand_name}.json').open('w') as f_out:
        json.dump(
            data, 
            f_out,
            indent=True,
        )

[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling AddHs()
[17:29:14] Molecule does not have explicit Hs. Consider calling 

### AF3 results

In [4]:
af3_output_folder = data_folder / 'glycan_modifications' / 'af3__lysozyme_pulldown'

results_df = process_af3_ligand_pulldown_results('Human_Lysozyme', af3_output_folder, run_posebusters=True)
results_df

folder  \
protein_name   ligand_name                                      
Human_Lysozyme LIG-PG-DAG-MUR  human_lysozyme_p61626__dag-mur   
               LIG-PG-NAG-MUR  human_lysozyme_p61626__nag-mur   
               LIG-PG-G-DAM      human_lysozyme_p61626__g-dam   
               LIG-PG-G-NAM      human_lysozyme_p61626__g-nam   
               LIG-PG-NAG-DAM  human_lysozyme_p61626__nag-dam   
               LIG-PG-DAG-DAM  human_lysozyme_p61626__dag-dam   
               LIG-PG-G-MUR      human_lysozyme_p61626__g-mur   
               LIG-PG-DAG-NAM  human_lysozyme_p61626__dag-nam   
               LIG-PG-NAG-NAM  human_lysozyme_p61626__nag-nam   

                                                                  structure_file  \
protein_name   ligand_name                                                         
Human_Lysozyme LIG-PG-DAG-MUR  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-MUR  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-DAM    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-NAM    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-DAM  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-DAM  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-MUR    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-NAM  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-NAM  /Users/rs1521/Documents/PG_modelling/glycan_mo...   

                                ptm  iptm  confidence  posebusters_score  \
protein_name   ligand_name                                                 
Human_Lysozyme LIG-PG-DAG-MUR  0.87  0.92       0.910                 20   
               LIG-PG-NAG-MUR  0.86  0.91       0.900                 20   
               LIG-PG-G-DAM    0.86  0.88       0.876                 20   
               LIG-PG-G-NAM    0.85  0.86       0.858                 20   
               LIG-PG-NAG-DAM  0.85  0.85       0.850                 20   
               LIG-PG-DAG-DAM  0.84  0.80       0.808                 20   
               LIG-PG-G-MUR    0.84  0.79       0.800                 20   
               LIG-PG-DAG-NAM  0.84  0.80       0.808                 19   
               LIG-PG-NAG-NAM  0.83  0.75       0.766                 19   

                              posebusters_errors  
protein_name   ligand_name                        
Human_Lysozyme LIG-PG-DAG-MUR               None  
               LIG-PG-NAG-MUR               None  
               LIG-PG-G-DAM                 None  
               LIG-PG-G-NAM                 None  
               LIG-PG-NAG-DAM               None  
               LIG-PG-DAG-DAM               None  
               LIG-PG-G-MUR                 None  
               LIG-PG-DAG-NAM    internal_energy  
               LIG-PG-NAG-NAM    internal_energy

# Glycans + stem

In [26]:
glycans_with_stem = pd.read_csv(data_folder / 'glycan_modifications' / 'glycans_with_stem.csv')
print(f'Number of fragments in subset: {len(glycans_with_stem):,}')
if 'ligand_name' not in glycans_with_stem.columns:
    glycans_with_stem['ligand_name'] = glycans_with_stem['Name'].apply(sanitize_ligand_name)
    glycans_with_stem.to_csv(data_folder / 'glycan_modifications' / 'glycans_with_stem.csv', index=False)

glycans_with_stem = glycans_with_stem.set_index('ligand_name')
glycans_with_stem

Number of fragments in subset: 9


,Name,Synonym,Formula,Monoisotopic Mass,Modifications,Degree Amidation,Degree Acetylation,Ontology,PGN Units,Glycan Units,...,Peptide,SMILES,INCHIKEY,clogP,RT,[M+H]+,[M+Na]+,[M+K]+,[M+2H]2+,[M+3H]3+
ligand_name,,,,,,,,,,,,,,,,,,,,,
G-NAM-AqKAA,(G)(NAM)-AqKAA,GlcN.MurNAc--Ala.γ-isoGln.Lys.Ala.Ala,C37H65N9O18,923.44476,NaN,1,1,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(O)OC(CO)C(OC2OC(CO)C(O)C(O)C2N)C1OC...,CEWPULLDOHGJFJ-UHFFFAOYSA-N,-7.9114,0,924.45204,946.43398,962.40792,462.72966,308.82220
G-DAM-AqKAA,(G)(DAM)-AqKAA,GlcN.MurNAcOAc--Ala.γ-isoGln.Lys.Ala.Ala,C39H67N9O19,965.45532,NaN,1,2,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(O)OC(COC(C)=O)C(OC2OC(CO)C(O)C(O)C2...,MBBOMWPDFVBEFO-UHFFFAOYSA-N,-7.3406,0,966.46260,988.44454,1004.41848,483.73494,322.82572
NAG-MUR-AqKAA,(NAG)(MUR)-AqKAA,GlcNAc.MurN--Ala.γ-isoGln.Lys.Ala.Ala,C37H65N9O18,923.44476,NaN,1,1,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)NC(C)C...,XHMRJLWLLVNRID-UHFFFAOYSA-N,-7.9114,0,924.45204,946.43398,962.40792,462.72966,308.82220
DAG-MUR-AqKAA,(DAG)(MUR)-AqKAA,GlcNAcOAc.MurN--Ala.γ-isoGln.Lys.Ala.Ala,C39H67N9O19,965.45532,NaN,1,2,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)NC(C)C...,BOQLSOQRBUSEDT-UHFFFAOYSA-N,-7.3406,0,966.46260,988.44454,1004.41848,483.73494,322.82572
NAG-NAM-AqKAA,(NAG)(NAM)-AqKAA,GlcNAc.MurNAc--Ala.γ-isoGln.Lys.Ala.Ala,C39H67N9O19,965.45532,NaN,1,2,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,MYHLLWFYGIKMKM-UHFFFAOYSA-N,-7.7340,0,966.46260,988.44454,1004.41848,483.73494,322.82572
DAG-NAM-AqKAA,(DAG)(NAM)-AqKAA,GlcNAcOAc.MurNAc--Ala.γ-isoGln.Lys.Ala.Ala,C41H69N9O20,1007.46589,NaN,1,3,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,QTFTZFLPZOTKNK-UHFFFAOYSA-N,-7.1632,0,1008.47317,1030.45511,1046.42905,504.74023,336.82924
NAG-DAM-AqKAA,(NAG)(DAM)-AqKAA,GlcNAc.MurNAcOAc--Ala.γ-isoGln.Lys.Ala.Ala,C41H69N9O20,1007.46589,NaN,1,3,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,ZXAGVKGKQYVLOI-UHFFFAOYSA-N,-7.1632,0,1008.47317,1030.45511,1046.42905,504.74023,336.82924
DAG-DAM-AqKAA,(DAG)(DAM)-AqKAA,GlcNAcOAc.MurNAcOAc--Ala.γ-isoGln.Lys.Ala.Ala,C43H71N9O21,1049.47645,NaN,1,4,G2--P5,1,2,...,AqKAA,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,BMSUWBVYWHQPBP-UHFFFAOYSA-N,-6.5924,0,1050.48373,1072.46567,1088.43961,525.74551,350.83276
G-MUR-AqKAA,(G)(MUR)-AqKAA,GlcN.MurN--Ala.γ-isoGln.Lys.Ala.Ala,C35H63N9O17,881.43419,NaN,1,0,G2--P5,1,2,...,AqKAA,CC(NC(=O)C(C)NC(=O)C(CCCCN)NC(=O)CCC(NC(=O)C(C...,PKMDYYVCWWROLE-UHFFFAOYSA-N,-8.0888,0,882.44147,904.42341,920.39735,441.72438,294.81868


In [28]:
glycans_with_stem.loc['NAG-MUR-AqKAA', 'SMILES']

'CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)NC(C)C(=O)NC(CCC(=O)NC(CCCCN)C(=O)NC(C)C(=O)NC(C)C(=O)O)C(N)=O)OC(CO)C(O)C1O'

In [20]:
ligand_folder_with_stem_af3 = data_folder / 'glycan_modifications' / 'af3__glycans_with_stem'
ligand_folder_with_stem_af3.mkdir(exist_ok=True)

ligand_folder_with_stem_protenix = data_folder / 'glycan_modifications' / 'protenix__glycans_with_stem'
ligand_folder_with_stem_protenix.mkdir(exist_ok=True)

for ligand_name, row in glycans_with_stem.iterrows():
    raw_name = row['Name']
    smiles = row['SMILES']

    mol = Chem.MolFromSmiles(smiles)
    try:
        mol = generate_conformation(mol)
    except ValueError:
        print(f'Error for ligand: {raw_name}')
        raise

    output_path = ligand_folder_with_stem_protenix / f'{ligand_name}.pdb'
    Chem.MolToPDBFile(mol, output_path.as_posix())

    ccd_code = f'LIG-PG-{ligand_name}'.replace('_', '-')
    ligand_seq = {
        'ligand': {
            'id': 'B',
            'ccdCodes': [ccd_code],
        }
    }
    try:
        ccd_data = generate_ccd_from_mol(mol, ccd_code)
    except ValueError:
        print(f'Error for ligand: {raw_name}')
        raise

    data = {
        'name': ligand_name,
        'sequences': [ligand_seq],
        'userCCD': ccd_data,
        'dialect': 'alphafold3',
        'version': 1,
    }
    with (ligand_folder_with_stem_af3 / f'{ligand_name}.json').open('w') as f_out:
        json.dump(
            data, 
            f_out,
            indent=True,
        )

[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling AddHs()
[23:25:25] Molecule does not have explicit Hs. Consider calling 

### Results AF3

In [22]:
af3_output_folder = data_folder / 'glycan_modifications' / 'af3__lysozyme_pulldown_with_stem'

results_df = process_af3_ligand_pulldown_results('Human_Lysozyme', af3_output_folder, run_posebusters=True)
results_df

structure_file  \
protein_name   ligand_name                                                               
Human_Lysozyme LIG-PG-DAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-NAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-MUR-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-DAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   

                                      ptm  iptm  confidence  \
protein_name   ligand_name                                    
Human_Lysozyme LIG-PG-DAG-MUR-AqKAA  0.79  0.65       0.678   
               LIG-PG-NAG-DAM-AqKAA  0.80  0.71       0.728   
               LIG-PG-NAG-NAM-AqKAA  0.82  0.75       0.764   
               LIG-PG-G-NAM-AqKAA    0.83  0.80       0.806   
               LIG-PG-DAG-DAM-AqKAA  0.79  0.67       0.694   
               LIG-PG-G-MUR-AqKAA    0.82  0.73       0.748   
               LIG-PG-G-DAM-AqKAA    0.82  0.77       0.780   
               LIG-PG-DAG-NAM-AqKAA  0.81  0.73       0.746   
               LIG-PG-NAG-MUR-AqKAA  0.79  0.63       0.662   

                                     posebusters_score  \
protein_name   ligand_name                               
Human_Lysozyme LIG-PG-DAG-MUR-AqKAA                 20   
               LIG-PG-NAG-DAM-AqKAA                 20   
               LIG-PG-NAG-NAM-AqKAA                 20   
               LIG-PG-G-NAM-AqKAA                   19   
               LIG-PG-DAG-DAM-AqKAA                 19   
               LIG-PG-G-MUR-AqKAA                   19   
               LIG-PG-G-DAM-AqKAA                   19   
               LIG-PG-DAG-NAM-AqKAA                 19   
               LIG-PG-NAG-MUR-AqKAA                 19   

                                              posebusters_errors  energy_ratio  
protein_name   ligand_name                                                      
Human_Lysozyme LIG-PG-DAG-MUR-AqKAA                         None           7.6  
               LIG-PG-NAG-DAM-AqKAA                         None           9.0  
               LIG-PG-NAG-NAM-AqKAA                         None          12.9  
               LIG-PG-G-NAM-AqKAA    minimum_distance_to_protein           8.8  
               LIG-PG-DAG-DAM-AqKAA  minimum_distance_to_protein           9.3  
               LIG-PG-G-MUR-AqKAA    minimum_distance_to_protein          10.8  
               LIG-PG-G-DAM-AqKAA    minimum_distance_to_protein          11.8  
               LIG-PG-DAG-NAM-AqKAA  minimum_distance_to_protein          25.8  
               LIG-PG-NAG-MUR-AqKAA  minimum_distance_to_protein          26.7

In [23]:
af3_output_folder = data_folder / 'glycan_modifications' / 'af3__lysozyme_pulldown_with_stem'
results_all = process_af3_ligand_pulldown_results('Human_Lysozyme', af3_output_folder, run_posebusters=True, score_all_sample=True)
results_all

structure_file  \
protein_name   ligand_name                                                               
Human_Lysozyme LIG-PG-DAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-DAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-MUR-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-G-NAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-DAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               LIG-PG-NAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   

                                      ptm  iptm  confidence  \
protein_name   ligand_name                                    
Human_Lysozyme LIG-PG-DAG-MUR-AqKAA  0.79  0.65       0.678   
               LIG-PG-G-DAM-AqKAA    0.81  0.75       0.762   
               LIG-PG-G-MUR-AqKAA    0.78  0.61       0.644   
               LIG-PG-G-NAM-AqKAA    0.82  0.74       0.756   
               LIG-PG-DAG-DAM-AqKAA  0.77  0.62       0.650   
               LIG-PG-NAG-MUR-AqKAA  0.76  0.55       0.592   
               LIG-PG-NAG-DAM-AqKAA  0.78  0.64       0.668   
               LIG-PG-DAG-NAM-AqKAA  0.81  0.72       0.738   
               LIG-PG-NAG-NAM-AqKAA  0.80  0.70       0.720   

                                     posebusters_score posebusters_errors  \
protein_name   ligand_name                                                  
Human_Lysozyme LIG-PG-DAG-MUR-AqKAA                 20               None   
               LIG-PG-G-DAM-AqKAA                   20               None   
               LIG-PG-G-MUR-AqKAA                   20               None   
               LIG-PG-G-NAM-AqKAA                   20               None   
               LIG-PG-DAG-DAM-AqKAA                 20               None   
               LIG-PG-NAG-MUR-AqKAA                 20               None   
               LIG-PG-NAG-DAM-AqKAA                 20               None   
               LIG-PG-DAG-NAM-AqKAA                 20               None   
               LIG-PG-NAG-NAM-AqKAA                 20               None   

                                     energy_ratio  
protein_name   ligand_name                         
Human_Lysozyme LIG-PG-DAG-MUR-AqKAA           7.6  
               LIG-PG-G-DAM-AqKAA             7.8  
               LIG-PG-G-MUR-AqKAA             7.9  
               LIG-PG-G-NAM-AqKAA             8.1  
               LIG-PG-DAG-DAM-AqKAA           8.1  
               LIG-PG-NAG-MUR-AqKAA           8.3  
               LIG-PG-NAG-DAM-AqKAA           8.4  
               LIG-PG-DAG-NAM-AqKAA           9.2  
               LIG-PG-NAG-NAM-AqKAA          11.6

In [24]:
results_all.loc[('Human_Lysozyme', 'LIG-PG-NAG-NAM-AqKAA'), 'structure_file']

'/Users/rs1521/Documents/PG_modelling/glycan_modifications/af3__lysozyme_pulldown_with_stem/human_lysozyme_p61626__nag-nam-aqkaa/seed-83_sample-2/model.cif'

In [25]:
results_all.loc[('Human_Lysozyme', 'LIG-PG-NAG-MUR-AqKAA'), 'structure_file']

'/Users/rs1521/Documents/PG_modelling/glycan_modifications/af3__lysozyme_pulldown_with_stem/human_lysozyme_p61626__nag-mur-aqkaa/seed-49_sample-4/model.cif'

### Results Protenix

In [10]:
protenix_lysozyme_folder_with_stem = data_folder / 'glycan_modifications' / 'protenix__lysozyme_pulldown_with_stem'
protenix_results_df = process_protenix_ligand_pulldown_results(
    'Human_Lysozyme', 
    protenix_lysozyme_folder_with_stem, 
    run_posebusters=True,
    score_all_sample=True,
)
protenix_results_df

structure_file  \
protein_name   ligand_name                                                        
Human_Lysozyme G-MUR-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               G-DAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               G-NAM-AqKAA    /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               DAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               DAG-NAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               DAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-MUR-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   
               NAG-DAM-AqKAA  /Users/rs1521/Documents/PG_modelling/glycan_mo...   

                                   ptm      iptm  confidence  \
protein_name   ligand_name                                     
Human_Lysozyme G-MUR-AqKAA    0.966671  0.976533       0.975   
               G-DAM-AqKAA    0.967807  0.978904       0.977   
               G-NAM-AqKAA    0.964892  0.979875       0.977   
               DAG-MUR-AqKAA  0.959234  0.966989       0.965   
               NAG-NAM-AqKAA  0.961475  0.972782       0.971   
               DAG-NAM-AqKAA  0.952837  0.960181       0.959   
               DAG-DAM-AqKAA  0.954563  0.950414       0.951   
               NAG-MUR-AqKAA  0.956946  0.965304       0.964   
               NAG-DAM-AqKAA  0.960006  0.969389       0.968   

                              posebusters_score posebusters_errors  \
protein_name   ligand_name                                           
Human_Lysozyme G-MUR-AqKAA                   20               None   
               G-DAM-AqKAA                   20               None   
               G-NAM-AqKAA                   20               None   
               DAG-MUR-AqKAA                 20               None   
               NAG-NAM-AqKAA                 20               None   
               DAG-NAM-AqKAA                 20               None   
               DAG-DAM-AqKAA                 20               None   
               NAG-MUR-AqKAA                 20               None   
               NAG-DAM-AqKAA                 20               None   

                              energy_ratio  
protein_name   ligand_name                  
Human_Lysozyme G-MUR-AqKAA             6.1  
               G-DAM-AqKAA             7.3  
               G-NAM-AqKAA             7.3  
               DAG-MUR-AqKAA           8.1  
               NAG-NAM-AqKAA           9.0  
               DAG-NAM-AqKAA           9.7  
               DAG-DAM-AqKAA           9.8  
               NAG-MUR-AqKAA          10.2  
               NAG-DAM-AqKAA          10.7

In [11]:
protenix_results_df.loc[('Human_Lysozyme', 'NAG-NAM-AqKAA'), 'structure_file']

'/Users/rs1521/Documents/PG_modelling/glycan_modifications/protenix__lysozyme_pulldown_with_stem/Human_Lysozyme_C_P61626__NAG-NAM-AqKAA/seed_735/predictions/Human_Lysozyme_C_P61626__NAG-NAM-AqKAA_seed_735_sample_2.cif'

In [12]:
protenix_results_df.loc[('Human_Lysozyme', 'NAG-MUR-AqKAA'), 'structure_file']

'/Users/rs1521/Documents/PG_modelling/glycan_modifications/protenix__lysozyme_pulldown_with_stem/Human_Lysozyme_C_P61626__NAG-MUR-AqKAA/seed_735/predictions/Human_Lysozyme_C_P61626__NAG-MUR-AqKAA_seed_735_sample_2.cif'

## PLIP

[Protein-Ligand Interaction Profiler (PLIP)](https://github.com/pharmai/plip)

In [14]:
import subprocess
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    pdb_path = Path(tmpdir) / 'structure.pdb'
    cif_path = protenix_results_df.loc[('Human_Lysozyme', 'NAG-NAM-AqKAA'), 'structure_file']

    subprocess.run(['obabel', '-icif', cif_path, '-opdb', '-O', pdb_path.as_posix()])

    subprocess.run(['plip', '-f', pdb_path.as_posix(), '-o', tmpdir, '-t'])

    with (Path(tmpdir) / 'report.txt').open() as f:
        content = f.read()
        print(content)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is Human_Lysozyme_C_P61626__NAG-NAM-AqKAA_seed_735_sample_2_predicted_by_protenix)

1 molecule converted


None


2025-02-06 17:31:45,098 [INFO] [plipcmd.py:134] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.4.0
2025-02-06 17:31:45,098 [INFO] [plipcmd.py:135] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2025-02-06 17:31:45,098 [INFO] [plipcmd.py:136] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2025-02-06 17:31:45,098 [INFO] [plipcmd.py:60] plip.plipcmd: starting analysis of structure.pdb
2025-02-06 17:31:45,288 [INFO] [plipcmd.py:175] plip.plipcmd: finished analysis, find the result files in /var/folders/xd/714yy8tn5vz90_wtnn7vglhc0000gq/T/tmpjkp_wsfe/


Prediction of noncovalent interactions for PDB structure STRUCTURE_PROTEIN
Created on 2025/02/06 using PLIP v2.4.0

If you are using PLIP in your work, please cite:
Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
Analysis was done on model 1.

l01:B:1 (l01) - SMALLMOLECULE
-----------------------------
Interacting chain(s): A


**Hydrophobic Interactions**
+-------+---------+----------+-----------+-------------+--------------+------+--------------+---------------+--------------------------+--------------------------+
| RESNR | RESTYPE | RESCHAIN | RESNR_LIG | RESTYPE_LIG | RESCHAIN_LIG | DIST | LIGCARBONIDX | PROTCARBONIDX | LIGCOO                   | PROTCOO                  | 
+=======+=========+==========+===========+=============+==============+======+==============+===============+==========================+==========================+
| 81    | TYR     | A  

## Boltz results

In [32]:
from src.pg_modelling.ligand_utils import run_pose_busters

boltz_res = run_pose_busters(
    Path('/Users/rs1521/Downloads/boltz_example2_model_0.cif'),
    'LIG',
    full_report=True,
).iloc[0]
boltz_res

mol_pred_loaded                        True
mol_cond_loaded                        True
sanitization                           True
inchi_convertible                      True
all_atoms_connected                    True
                                      ...  
volume_overlap_inorganic_cofactors      NaN
volume_overlap_waters                   NaN
score                                    19
max_score                                20
perfect_score                         False
Name: LIG, Length: 108, dtype: object

In [33]:
for name, value in boltz_res.items():
    print(name, value)

mol_pred_loaded True
mol_cond_loaded True
sanitization True
inchi_convertible True
all_atoms_connected True
bond_lengths True
bond_angles True
internal_steric_clash True
aromatic_ring_flatness True
double_bond_flatness True
internal_energy True
protein-ligand_maximum_distance True
minimum_distance_to_protein False
minimum_distance_to_organic_cofactors True
minimum_distance_to_inorganic_cofactors True
minimum_distance_to_waters True
volume_overlap_with_protein True
volume_overlap_with_organic_cofactors True
volume_overlap_with_inorganic_cofactors True
volume_overlap_with_waters True
mol_true_loaded False
passes_valence_checks True
passes_kekulization True
number_bonds 65
shortest_bond_relative_length 0.8606049641397592
longest_bond_relative_length 1.0634696709525246
number_short_outlier_bonds 0
number_long_outlier_bonds 0
number_angles 90
most_extreme_relative_angle 1.0634377312420267
number_outlier_angles 0
number_noncov_pairs 1861
shortest_noncovalent_relative_distance 0.9394566547895

## Pipeline results

In [47]:
scores_df = pd.read_csv('/Users/rs1521/Documents/PG_modelling/Lysozyme/protenix/scores.csv', index_col=['protein_name', 'ligand_name'])
scores_df.head()

seed  sample  \
protein_name            ligand_name                   
Human_Lysozyme_C_P61626 NAG-NAM-AqKAA   558       0   
                        NAG-NAM-AqKAA   558       4   
                        NAG-NAM-AqKAA   558       1   
                        NAG-NAM-AqKAA   966       1   
                        NAG-NAM-AqKAA   187       1   

                                                                          structure_file  \
protein_name            ligand_name                                                        
Human_Lysozyme_C_P61626 NAG-NAM-AqKAA  /gpfs/home/rs1521/PG/Human_Lysozyme_C_P61626/m...   
                        NAG-NAM-AqKAA  /gpfs/home/rs1521/PG/Human_Lysozyme_C_P61626/m...   
                        NAG-NAM-AqKAA  /gpfs/home/rs1521/PG/Human_Lysozyme_C_P61626/m...   
                        NAG-NAM-AqKAA  /gpfs/home/rs1521/PG/Human_Lysozyme_C_P61626/m...   
                        NAG-NAM-AqKAA  /gpfs/home/rs1521/PG/Human_Lysozyme_C_P61626/m...   

                                            ptm      iptm  confidence  \
protein_name            ligand_name                                     
Human_Lysozyme_C_P61626 NAG-NAM-AqKAA  0.964197  0.976633       0.974   
                        NAG-NAM-AqKAA  0.963599  0.975112       0.973   
                        NAG-NAM-AqKAA  0.961727  0.976360       0.973   
                        NAG-NAM-AqKAA  0.959808  0.974470       0.972   
                        NAG-NAM-AqKAA  0.960942  0.975195       0.972   

                                       posebusters_score  energy_ratio  \
protein_name            ligand_name                                      
Human_Lysozyme_C_P61626 NAG-NAM-AqKAA                 20          13.3   
                        NAG-NAM-AqKAA                 20          11.7   
                        NAG-NAM-AqKAA                 20          20.5   
                        NAG-NAM-AqKAA                 20          11.2   
                        NAG-NAM-AqKAA                 20          13.5   

                                      posebusters_errors  
protein_name            ligand_name                       
Human_Lysozyme_C_P61626 NAG-NAM-AqKAA                NaN  
                        NAG-NAM-AqKAA                NaN  
                        NAG-NAM-AqKAA                NaN  
                        NAG-NAM-AqKAA                NaN  
                        NAG-NAM-AqKAA                NaN

In [50]:
scores_df.reset_index()[[
    'protein_name',
    'ligand_name',
    'ptm',
    'iptm',
    'posebusters_score',
    'confidence',
    'energy_ratio',
]].groupby([
    'protein_name', 
    'ligand_name',
]).mean().sort_values(
    [
        'posebusters_score',
        'confidence',
        'energy_ratio',
    ],
    ascending=[False, False, True],
)

ptm      iptm  posebusters_score  \
protein_name            ligand_name                                            
Human_Lysozyme_C_P61626 NAG-NAM-AqKAA  0.960754  0.973138          19.600000   
                        NAG-MUR-AqKAA  0.959141  0.965958          19.466667   

                                       confidence  energy_ratio  
protein_name            ligand_name                              
Human_Lysozyme_C_P61626 NAG-NAM-AqKAA    0.970533     14.546667  
                        NAG-MUR-AqKAA    0.964533     17.320000

In [5]:
scores_df = pd.read_csv(
    '/Users/rs1521/Documents/PG_modelling/Lysozyme/boltz/scores.csv', 
    index_col=['protein_name', 'ligand_name'],
)
scores_df.tail()

seed  sample  \
protein_name            ligand_name                   
Human_Lysozyme_C_P61626 NAG-MUR-AqKAA   822       4   
                        NAG-MUR-AqKAA   931       4   
                        NAG-NAM-AqKAA   436       4   
                        NAG-MUR-AqKAA   822       2   
                        NAG-NAM-AqKAA   822       3   

                                                                          structure_file  \
protein_name            ligand_name                                                        
Human_Lysozyme_C_P61626 NAG-MUR-AqKAA  /gpfs/home/rs1521/PG/Human_Lysozyme_C_P61626/m...   
                        NAG-MUR-AqKAA  /gpfs/home/rs1521/PG/Human_Lysozyme_C_P61626/m...   
                        NAG-NAM-AqKAA  /gpfs/home/rs1521/PG/Human_Lysozyme_C_P61626/m...   
                        NAG-MUR-AqKAA  /gpfs/home/rs1521/PG/Human_Lysozyme_C_P61626/m...   
                        NAG-NAM-AqKAA  /gpfs/home/rs1521/PG/Human_Lysozyme_C_P61626/m...   

                                            ptm      iptm  confidence  \
protein_name            ligand_name                                     
Human_Lysozyme_C_P61626 NAG-MUR-AqKAA  0.743023  0.439899       0.501   
                        NAG-MUR-AqKAA  0.739423  0.427251       0.490   
                        NAG-NAM-AqKAA  0.771610  0.539508       0.586   
                        NAG-MUR-AqKAA  0.757589  0.458889       0.519   
                        NAG-NAM-AqKAA  0.764222  0.516305       0.566   

                                       posebusters_score  energy_ratio  \
protein_name            ligand_name                                      
Human_Lysozyme_C_P61626 NAG-MUR-AqKAA               18.0   122771329.6   
                        NAG-MUR-AqKAA               18.0        1419.3   
                        NAG-NAM-AqKAA               17.0    29363576.5   
                        NAG-MUR-AqKAA               17.0        9350.1   
                        NAG-NAM-AqKAA                NaN           NaN   

                                                                      posebusters_errors  
protein_name            ligand_name                                                       
Human_Lysozyme_C_P61626 NAG-MUR-AqKAA        internal_energy,minimum_distance_to_protein  
                        NAG-MUR-AqKAA        internal_energy,minimum_distance_to_protein  
                        NAG-NAM-AqKAA  internal_steric_clash,internal_energy,minimum_...  
                        NAG-MUR-AqKAA  internal_steric_clash,internal_energy,minimum_...  
                        NAG-NAM-AqKAA                                                NaN

In [3]:
from src.pg_modelling.ligand_utils import run_pose_busters, POSEBUSTERS_CHECKS

res_df = run_pose_busters(
    Path(
        '/Users/rs1521/Documents/PG_modelling/Lysozyme/protenix/results/Human_Lysozyme_C_P61626__NAG-NAM-AqKAA/seed_966/predictions/Human_Lysozyme_C_P61626__NAG-NAM-AqKAA_seed_966_sample_2.cif'
    ),
    'LIG2',
)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is Human_Lysozyme_C_P61626__NAG-NAM-AqKAA_seed_966_sample_2_predicted_by_protenix)

1 molecule converted
1 molecule converted


In [4]:
res_df[POSEBUSTERS_CHECKS].iloc[0]

mol_pred_loaded                            True
mol_cond_loaded                            True
sanitization                               True
inchi_convertible                          True
all_atoms_connected                        True
bond_lengths                               True
bond_angles                                True
internal_steric_clash                      True
aromatic_ring_flatness                     True
double_bond_flatness                       True
internal_energy                            True
protein-ligand_maximum_distance            True
minimum_distance_to_protein                True
minimum_distance_to_organic_cofactors      True
minimum_distance_to_inorganic_cofactors    True
minimum_distance_to_waters                 True
volume_overlap_with_protein                True
volume_overlap_with_organic_cofactors      True
volume_overlap_with_inorganic_cofactors    True
volume_overlap_with_waters                 True
Name: LIG2, dtype: bool